In [1]:
import os
import gc
import sys
import pandas as pd
import numpy as np
import ast
os.environ['PYSPARK_PYTHON'] = sys.executable 
os.environ['PYSPARK_DRIVER_PYTHON'] = sys.executable
os.environ["PYARROW_IGNORE_TIMEZONE"] = '1' 

# imports para trabajar con spark en local
import findspark
findspark.init()
from pyspark.sql import SparkSession
from pyspark.sql.utils import AnalysisException
from pyspark.sql.types import IntegerType,StructField,StructType,StringType,LongType,MapType
from pyspark.sql import functions as F
import pyspark.pandas as ps

# imports para trabajar con GCP
from google.oauth2 import service_account
from google.cloud import translate_v2 as translate




# Credenciales para API's de GCP
SERVICE_ACCOUNT_FILE = '../../credentials/fiery-protocol-399500-f2566dd92ef4.json'
creds = service_account.Credentials.from_service_account_file(
        SERVICE_ACCOUNT_FILE)
translate_client = translate.Client(credentials=creds)


# Iniciamos sesión de spark.
SparkSession.stop(spk)
spk = SparkSession.builder.appName("PySpark Transformations to Populate our Data Warehouse") \
        .master("local[5]") \
        .config("spark.executor.memory","2g") \
        .config("spark.driver.memory","2g") \
        .config("spark.speculation","false") \
        .config("parquet.enable.summary-metadata", "false") \
        .config("mapreduce.fileoutputcommitter.marksuccessfuljobs", "false") \
        .getOrCreate()
ps.options.compute.ops_on_diff_frames = True
spk

In [2]:
schema = StructType([
    StructField('user_id',StringType(),False),
    StructField('name',StringType(),True),
    StructField('time',LongType(),True),
    StructField('rating',IntegerType(),True),
    StructField('text',StringType(),True),
    StructField('resp',MapType(StringType(),StringType()),False),
    StructField('gmap_id',StringType(),False)
])


i = 1
df_list = []
while True:
    try:

        # Leemos los archivos en un SPARK Data Frame para poder acceder directamente a GCS
        sdf = spk.read.schema(schema).json(f'../../data/Google Maps/estados/review-California/{i}.json')[['gmap_id', 'user_id', 'name', 'time', 'rating', 'text', 'resp']].cache()
        sdf.selectExpr('cast(user_id as int) user_id').cache()
        sdf = sdf.withColumn('tiempo_respuesta',F.col('resp').getField('time')).cache()
        sdf = sdf.withColumn('tiempo_respuesta',F.col('tiempo_respuesta').cast(LongType())).cache()
        sdf = sdf.withColumn('respuesta',F.col('resp').getField('text')).sort('user_id').cache()
        sdf = sdf.withColumn('time',F.col('time').cast(LongType())).cache()
        sdf = sdf[['gmap_id','user_id','name','time','rating','text','tiempo_respuesta','respuesta']].cache()
        # Transformamos el dataframe de respuestas a pandas para realizar las siguientes operaciones
        # respuestas = respuestas.toPandas()
        # respuestas['lenguaje_respuesta'] = pd.Series([],dtype=str)
        # for i in range(len(respuestas)):
        #     translation = translate_client.translate(respuestas.loc[i,'respuesta'], target_language='en')
        #     if translation['detectedSourceLanguage'] == 'en':
        #         respuestas.loc[i,'lenguaje_respuesta'] = 'en'
        #     else:
        #         respuestas.loc[i,'respuesta'] = translation['translatedText']
        #         respuestas.loc[i,'lenguaje_respuesta'] = translation['detectedSourceLanguage']


        # PANDAS API Data Frame: Paso intermedio para trabajar con los métodos de pandas pero con la potencia de spark, posteriormente guardaremos los datos en BQ después de 
        # las transformaciones...
        psdf = sdf.pandas_api()
        # sdf.unpersist()
        # del sdf
        # gc.collect()
        # psdf = psdf[['gmap_id','user_id','name','time','rating','text','tiempo_respuesta','respuesta']].reset_index(drop=True).spark.cache()
        psdf['estado'] = 'California' # state
        df_list.append(psdf)
        i += 1
    except AnalysisException:
        break

psdf = ps.concat(df_list,axis=0)
psdf = psdf.drop_duplicates().reset_index().spark.cache()   # 2,700,000 registros antes de drop_duplicates()

del df_list
gc.collect()

sdft = psdf.to_spark().cache()
csdf = sdft.fillna('nan')
srespuestas = csdf[csdf.respuesta != 'nan'][['index','gmap_id','user_id','tiempo_respuesta','respuesta']].cache()
srespuestas_gytrans = srespuestas.where(srespuestas.respuesta.contains('(Translated by Google)')).cache()
srespuestas_gntrans = srespuestas.where(srespuestas.respuesta.contains('(Translated by Google)') == False).cache()
del srespuestas
del csdf
gc.collect()
# psdf.to_csv('../../data/Google Maps/clean_test/estados/california')  # 59.8s master('local[6]')
# sdft.coalesce(1).write.mode('overwrite').format('csv').save('../../data/Google Maps/clean_test/estados/california')  # 56.2s, 1m 8s master('local[6]') # 57.6s, 1m 0.4s master('local[5]') # 1m 42.1s master('local[3]') # 1m 17s master('local[9]') # 1m 52s master('local[*]') # 1m 5.1s master('local[7]')
print(f'pyspark.pandas data frame persisted')  # 2.3s master(local[5]) # 2.8s master(local[1]) # 4.8s master(local[*])

pyspark.pandas data frame persisted


C:\spark-3.5.0\python\pyspark\pandas\utils.py:1016: PandasAPIOnSparkAdviceWarning: If `index_col` is not specified for `to_spark`, the existing index is lost when converting to Spark DataFrame.
  warnings.warn(message, PandasAPIOnSparkAdviceWarning)


In [253]:
sdft.show()

+------+--------------------+--------------------+---------------+-------------+------+--------------------+----------------+--------------------+----------+
| index|             gmap_id|             user_id|           name|         time|rating|                text|tiempo_respuesta|           respuesta|    estado|
+------+--------------------+--------------------+---------------+-------------+------+--------------------+----------------+--------------------+----------+
| 23183|0x0:0x13154eabf70...|10363023463792407...|   Evelyn Smith|1597486754434|     5|        Easy access.|            NULL|                NULL|California|
| 35641|0x0:0x13154eabf70...|10461531724534491...|     Brett Pyle|1560665142117|     5|        Fun airport!|            NULL|                NULL|California|
| 84147|0x0:0x13154eabf70...|11042059297073811...|     jeff brown|1584485924747|     5|                NULL|            NULL|                NULL|California|
| 85571|0x0:0x13154eabf70...|11059108636833709...|  

In [237]:
sdft.printSchema()

root
 |-- index: long (nullable = false)
 |-- gmap_id: string (nullable = true)
 |-- user_id: string (nullable = true)
 |-- name: string (nullable = true)
 |-- time: long (nullable = true)
 |-- rating: integer (nullable = true)
 |-- text: string (nullable = true)
 |-- tiempo_respuesta: long (nullable = true)
 |-- respuesta: string (nullable = true)
 |-- estado: string (nullable = false)



In [238]:
sdft.count()

2624757

In [239]:
srespuestas_gytrans.count()

4744

In [240]:
srespuestas_gntrans.count()

233713

In [3]:
prespuestas = srespuestas_gntrans.toPandas()
# psrespuestas = srespuestas_gntrans.pandas_api()

In [4]:
prespuestas.head(2)

,index,gmap_id,user_id,tiempo_respuesta,respuesta
0,102056,0x0:0xdf872b4788fc688b,112495125573974647398,1574524668849,Thank you. We appreciate your support
1,141203,0x0:0xdf872b4788fc688b,117364149439115386407,1546802390681,Thank you Gary!!!!


In [5]:
prespuestas.count()

index               233713
gmap_id             233713
user_id             233713
tiempo_respuesta    233713
respuesta           233713
dtype: int64

In [9]:
prespuestas = prespuestas[['gmap_id','user_id','respuesta','tiempo_respuesta']]
prespuestas[prespuestas.respuesta.str.contains('Thank you Gary!!!!')].index.item()

1

In [24]:
translate_client.get_languages('hola')

BadRequest: 400 GET https://translation.googleapis.com/language/translate/v2/languages?target=hola&prettyPrint=false: Target language is invalid. [{'@type': 'type.googleapis.com/google.rpc.BadRequest', 'fieldViolations': [{'field': 'target', 'description': 'Target language: hola'}]}]

In [25]:
def translate_resp(x):
    
    global idx_noten_apply, resp_noten_apply
    idx_noten_apply = []
    resp_noten_apply = []
    
    # imports para trabajar con GCP
    from google.oauth2 import service_account
    from google.cloud import translate_v2 as translate
    # Credenciales para API's de GCP
    SERVICE_ACCOUNT_FILE = '../../credentials/fiery-protocol-399500-f2566dd92ef4.json'
    creds = service_account.Credentials.from_service_account_file(
            SERVICE_ACCOUNT_FILE)
    translate_client = translate.Client(credentials=creds)
    
    idx = 0
    lista = list(prespuestas[prespuestas.respuesta.str.contains(x,regex=False)].index)
    translation = translate_client.translate(x,target_language='en')
    idioma = translation['detectedSourceLanguage']
    if len(lista) == 1:
        idx = prespuestas[prespuestas.respuesta.str.contains(x,regex=False)].index.item()
        if idx % 50 == 0:
            print(idx)
    if idioma != 'en':
        print(x,idioma,translation['translatedText'])
        idx_noten_apply.append(idx)
        resp_noten_apply.append(x)
        try:
            print(idx)
        except NameError:
            print(f'Ya encontramos esta frase antes, los índices son: {lista}')
        return translation['translatedText']
    return x

In [12]:
translate_prueba_pdf = prespuestas.respuesta[:2380].apply(translate_resp)  # 19.9s master(local[5])
translate_prueba_pdf

50
150
200
250
300
350
Thank you very much, Vikash Lakhani. mr Thank you very much, Vikash Lakhani.
384
400
450
Thanks Myka! fil Thanks Myka!
477
500
550
650
700
750
800
850
950
Thanks,  Kareem! gu Thanks Kareem!
975
1000
1100
1150
1200
1300
1400
1450
1500
1550
1600
1650
1700
1750
1800
xxxx ht xxxx
0
1850
1900
1950
2050
2100
Thanks Sanne! ;) da Thanks Sanne! ;)
2139
2150
2200
2300
2350


0                   Thank you. We appreciate your support
1                                      Thank you Gary!!!!
2       Tony, our sincere apologies you had this exper...
3       Thank you very much! You can always come back ...
4       Thank you Jeffery for taking time to review us...
                              ...                        
2375    Hi AM, thank you for your review. We are sorry...
2376    Thank you for your review! Our associates stri...
2377                        Santosh, we love seeing this!
2378    That's wonderful to hear, Darrell! We sure are...
2379              Igor, we appreciate your 5-star rating!
Name: respuesta, Length: 2380, dtype: object

In [14]:
resp_noten_apply

['Thank you very much, Vikash Lakhani.',
 'Thanks Myka!',
 'Thanks,  Kareem!',
 'xxxx',
 'Thanks Sanne! ;)']

In [21]:
idx_noten_loop = []
resp_noten_loop = []
lang_noten_loop = []
for i in range(len(prespuestas)):  #234m 8.4s total de registros 233k+
    if i % 100 == 0:
        print(i)
    respuesta = prespuestas.loc[i,'respuesta']
    idioma = translate_client.translate(respuesta)['detectedSourceLanguage']
    if idioma != 'en':
        # print(i,respuesta,idioma,translation['translatedText'])
        idx_noten_loop.append(i)
        resp_noten_loop.append(respuesta)
        lang_noten_loop.append(idioma)

0
384 Thank you very much, Vikash Lakhani. mr Thank you very much, Vikash Lakhani.
477 Thanks Myka! fil Thanks Myka!
975 Thanks,  Kareem! gu Thanks Kareem!
1000
1834 xxxx ht xxxx
2000
2139 Thanks Sanne! ;) da Thanks Sanne! ;)
3000
3475 Thanks, Mikki. ta Thanks, Mikki.
4000
4336 ThankS Schad! de Thanks Shame!
4404 Thank you Julito fil Thank you Julito
4676 Thank you
Annie Ngai. lus Thank you Annie Ngai.
4677 Thank you Betsaida Valdovinos-Ceja es Thank you Betsaida Valdovinos-Ceja
5000
6000
6236 Thanks Omkar! hi Thanks Omkar!
6237 Thanks Yuxuan! zh Thanks Yuxuan!
6254 Thanks Dayana! hi Thanks Dayana!
6260 Thanks Dassa! gu Thanks Dassa!
6286 Thanks Binh! vi Thanks Binh!
6601 Thanks Shadee! :) gu Thanks Shadee! :)
7000
7219 Thank you Luis Alvarado! es Thank you Luis Alvarado!
7377 Thank you for the 5-Star rating! We are glad to hear you enjoyed your experience at Quiznos! Please come back and visit again!.

Estimado Jesus Diaz, gracias por su recomendacion. Nos alegra saber que te gusto, p

In [26]:
respuestas_idxs = list(zip(resp_noten_loop,idx_noten_loop))
len(respuestas_idxs)

324

In [28]:
stranslate_resp = F.udf(lambda x: translate_resp(x),StringType()) # UDF
srespuestas_gntrans.withColumn('respuesta',stranslate_resp(F.col('respuesta'))).show() # UDF Aplicado a una columna.

PythonException: 
  An exception was thrown from the Python worker. Please see the stack trace below.
Traceback (most recent call last):
  File "C:\Users\tinma\AppData\Local\Temp\ipykernel_2664\2490335977.py", line 1, in <lambda>
  File "C:\Users\tinma\AppData\Local\Temp\ipykernel_2664\853709964.py", line 18, in translate_resp
  File "c:\Users\tinma\AppData\Local\Programs\Python\Python310\lib\site-packages\google\cloud\translate_v2\client.py", line 266, in translate
    response = self._connection.api_request(method="POST", path="", data=data)
  File "c:\Users\tinma\AppData\Local\Programs\Python\Python310\lib\site-packages\google\cloud\_http\__init__.py", line 494, in api_request
    raise exceptions.from_http_response(response)
google.api_core.exceptions.Forbidden: 403 POST https://translation.googleapis.com/language/translate/v2?prettyPrint=false: User Rate Limit Exceeded


In [193]:
srespuestas_gntrans.show(5)

+------+--------------------+--------------------+----------------+--------------------+
| index|             gmap_id|             user_id|tiempo_respuesta|           respuesta|
+------+--------------------+--------------------+----------------+--------------------+
|102056|0x0:0xdf872b4788f...|11249512557397464...|   1574524668849|Thank you. We app...|
|141203|0x0:0xdf872b4788f...|11736414943911538...|   1546802390681|  Thank you Gary!!!!|
|147824|0x0:0xdf872b4788f...|11818319885363239...|   1519404556470|Tony, our sincere...|
| 54382|0x14e179060f0442e...|10687919537608056...|   1548177848059|Thank you very mu...|
| 18486|0x14e19dbfedc0c44...|10251236466311407...|   1442083874424|Thank you Jeffery...|
+------+--------------------+--------------------+----------------+--------------------+
only showing top 5 rows



In [169]:
cprespuestas = prespuestas[:23846] # 23846 registros en 25m 3.4s
cprespuestas['respuesta'] = cprespuestas.respuesta.apply(translate_resp)
translate_prueba_pdf

0        None
1        None
2        None
3        None
4        None
         ... 
23841    None
23842    None
23843    None
23844    None
23845    None
Name: respuesta, Length: 23846, dtype: object

In [ ]:
j = 0
idx_notenglish = []
for x in prespuestas.respuesta[:23846]:
    if j % 100 == 0:
        print(i)
    translation = translate_client.translate(x)
    idioma = translation['detectedSourceLanguage']
    if idioma != 'en':
        idx_notenglish.append(i)
    j += 1

In [79]:
prespuestas['lenguaje_respuesta'] = pd.Series([],dtype=str)
for i in range(len(prespuestas)):   # 3m 3.4s para el 1% de los datos.
    if i % 100 == 0:
        print(i)
    translation = translate_client.translate(prespuestas.loc[i,'respuesta'], target_language='en')
    if translation['detectedSourceLanguage'] == 'en':
        prespuestas.loc[i,'lenguaje_respuesta'] = 'en'
    else:
        prespuestas.loc[i,'respuesta'] = translation['translatedText']
        prespuestas.loc[i,'lenguaje_respuesta'] = translation['detectedSourceLanguage']
prespuestas

0
100
200
300
400
500
600
700
800
900
1000
1100
1200
1300
1400
1500
1600
1700
1800
1900
2000
2100
2200
2300
2400
2500
2600
2700
2800
2900
3000


KeyboardInterrupt: 

In [9]:
translate_client.detect_language('hola cómo estás')

{'confidence': 1, 'language': 'es', 'input': 'hola cómo estás'}

In [11]:
translate_client.translate('hola cómo estás', target_language='en')

{'translatedText': 'Hi, how are you',
 'detectedSourceLanguage': 'es',
 'input': 'hola cómo estás'}

In [70]:
translate_client.translate('Hi, hello, how are you?', target_language='en')

{'translatedText': 'Hi, hello, how are you?',
 'detectedSourceLanguage': 'en',
 'input': 'Hi, hello, how are you?'}

In [38]:
sdf.count()

2700000

In [41]:
psdf.count()

index      2700000
gmap_id    2700000
user_id    2700000
name       2700000
time       2700000
rating     2700000
text       1529036
resp        245169
dtype: int64

In [103]:
languages_psdf = ps.DataFrame(translate_client.get_languages()).sort_values('name')
languages_psdf.head(2)

,language,name
0,af,Afrikaans
1,sq,Albanian


In [104]:
psdf.resp.loc[66]

'{"time":1629166810138,"text":"现在接线员是很好的英语"}'

In [54]:
psdf.resp.loc[66][30:-2]

'现在接线员是很好的英语'

In [ ]:
psdf['text'] = psdf.text.fillna('nan')
sdf = sdf.withColumn("resp", F.to_json("resp"))
respuestas = sdf.pandas_api()
respuestas = respuestas[['resp']]


In [185]:
psdf['text'] = psdf.text.fillna('nan')
sdf = sdf.withColumn("resp", F.to_json("resp"))
respuestas = sdf.pandas_api()
respuestas = respuestas[['resp']]
respuestas.spark.cache()
respuestas['resp'] = respuestas.text.fillna('nan')
respuestas = respuestas.resp != 'nan'
resp_idx = sorted(respuestas.index.tolist())
print(respuestas.head(2))
resp_idx[:10]

AttributeError: 'DataFrame' object has no attribute 'text'

In [77]:
psdf.resp.loc[66][:30] + f'{translation["translatedText"]}."' + '}'

'{"time":1629166810138,"text":"The operator now speaks very good English."}'

In [79]:
ast.literal_eval(psdf.resp.loc[66][:30] + f'{translation["translatedText"]}."' + '}')

{'time': 1629166810138, 'text': 'The operator now speaks very good English.'}

In [144]:
respuestas.loc[30,'resp']

'{"time":1631072651706,"text":"Thanks so much for your business and for taking the time to post a great review. Glad to hear our crew was so helpful and friendly and that we were able to get the family outfitted quickly and efficiently. Look forward to hearing from you in the future."}'

In [ ]:
for i in range(len(respuestas)):
    respuestas['resp'].apply(lambda x: ast.literal_eval(x))
    respuestas.loc[i,'resp_time'] = respuestas.loc[i,'resp']['time']
    respuestas.loc[i,'resp_text'] = respuestas.loc[i,'resp']['text']
respuestas

In [ ]:
PROJECT_ID = 'fiery-protocol-399500'
STATES = ['California'] #,'Texas'] # ,'New_York','Colorado','Georgia']
schema = StructType([
    StructField('user_id',StringType(),False),
    StructField('name',StringType(),True),
    StructField('time',LongType(),True),
    StructField('rating',IntegerType(),True),
    StructField('text',StringType(),True),
    StructField('resp',StringType(),False),
    StructField('gmap_id',StringType(),False)
])



psdfx = ps.DataFrame(columns=['gmap_id','user_id','name','time','text','rating','resp_time','resp_text'])
for state in STATES:
    i = 1
    df_list = []
    while True:
        try:
            # Leemos los archivos en un SPARK Data Frame para poder acceder directamente a GCS
            sdf = spk.read.schema(schema).json(f'../data/Google Maps/estados/review-{state}/{i}.json')[['user_id','name','time','rating','text','resp','gmap_id']].cache()
            sdf.selectExpr('cast(user_id as int) user_id')
            sdf.selectExpr('cast(null as map<string,string>) resp')
            # PANDAS API Data Frame: Paso intermedio para trabajar con los métodos de pandas pero con la potencia de spark, posteriormente guardaremos los datos en BQ después de 
            # las transformaciones...
            # sdf.count()
            psdf = sdf.pandas_api().spark.cache()
            sdf.unpersist()
            del sdf
            gc.collect()
            # psdf['time'] = ps.to_datetime(psdf['time'],unit='ms')
            psdf['estado'] = state
            psdf['resp'] = psdf.resp.fillna('nan')
            psdf['text'] = psdf.text.fillna('nan')
            df_list.append(psdf)
            i += 1
        except AnalysisException:
            break

    psdf = ps.concat(df_list,axis=0)
    psdf = psdf.reset_index(drop=True)
    del df_list
    psdf.spark.cache()
    print(f'pyspark.pandas data frame persisted - {state}')





#     # Generamos el primer grupo de transformaciones para los datos de las reviews de Maps en PANDAS API. Queda la metadata y los archivos de Yelp.
#     psdf['resp_time'] = ps.Series([],dtype='int64')
#     print('serie resp_time creada')
#     psdf['resp_text'] = ps.Series([],dtype='str')
#     psdf.spark.cache()
#     print('serie resp_text creada')
#     for i in range(len(psdf)):
#         print(i)
#         if type(psdf.loc[i,'resp']) == dict:
#             psdf.loc[i,'resp_time'] = psdf.loc[i,'resp']['time']
#             psdf.loc[i,'resp_text'] = psdf.loc[i,'resp']['text']
#         else:
#             pass
#     psdf.resp_time = psdf.resp_time.fillna(0).astype('int64')
#     psdf.resp_text = psdf.resp_text.fillna('')
#     psdf = psdf[['gmap_id','user_id','name','time','text','rating','resp_time','resp_text']]
#     psdf.spark.cache()
    
#     # Aquí concatenamos todos los archivos del estado en curso a los demás estados, para obtener una tabla total de estados.
#     psdfx = ps.concat(psdf,axis=0)
#     psdf.spark.unpersist()
#     del psdf
#     gc.collect()
#     print('pyspark.pandas data frame unpersisted and deleted')

# # Convertimos el dataframe de Pandas API on Spark a un dataframe de Spark
# sdf = psdfx.to_spark()
# del psdfx
# gc.collect()

# # Guardamos las tablas concatenadas en archivos .json en GCS.
# sdf.write.mode('overwrite').format('csv').save(f'../data/Google Maps/clean_test/estados/all_tables')